### Methods

#### Models

To answer the proposed questions we use a 1-d transport model ([Yakubov et al., 2019]) and a biogeochemical model ([Yakubov]).
In addition to the available parametrization, we add some supplementary functionality to the biogeochemical model.
The updated biogeochemical model operates with the same state variables as previously:

autotrophs, $\text{Phy}\ \lbrack\text{mg C m}^{- 3}\rbrack$;
heterotrophs, $\text{Het}\ \lbrack\text{mg C m}^{- 3}\rbrack$;
$\text{NH}_{4}^{+}\ \lbrack\text{mM N m}^{- 3}\rbrack$;
$\text{NO}_{2}^{-}\ \lbrack\text{mM N m}^{- 3}\rbrack$;
$\text{NO}_{3}^{-}\ \lbrack\text{mM N m}^{- 3}\rbrack$;
$\text{Si}\ \lbrack\text{mM Si m}^{- 3}\rbrack$;
$\text{PO}_{4}^{3 -}\ \lbrack\text{mM P m}^{- 3}\rbrack$;
$\text{O}_{2}\ \lbrack\text{mM O}_{2}\ \text{m}^{- 3}\rbrack$;
$\text{SO}_{4}^{2 -}\ \lbrack\text{mM S m}^{- 3}\rbrack$;
$\text{S}_{2}\text{O}_{3}^{2 -}\ \lbrack\text{mM S m}^{- 3}\rbrack$;
$\text{S}^{0}\ \lbrack\text{mM S m}^{- 3}\rbrack$;
$\text{H}_{2}\text{S}\ \lbrack\text{mM S m}^{- 3}\rbrack$;
dissolved inorganic carbon, $\text{DIC}\ \lbrack\text{mM C m}^{- 3}\rbrack$;
particulate OM ($\text{POM}\ \lbrack\text{mg C m}^{- 3}\rbrack$);
dissolved OM ($\text{DOM}\ \lbrack\text{mg C m}^{- 3}\rbrack$);
$\text{TA}\ \lbrack\text{mM}\ \text{m}^{- 3}\rbrack$.

$\text{POM}$ and $\text{DOM}$ have a Redfield stoichiometry of $106\text{C}:16\text{N}:1\text{P}$.

[Yakubov et al., 2019]: https://doi.org/10.3390/w11081582
[Yakubov]: https://github.com/limash/Alkalinity_in_the_Wadden_Sea

Further, the current biogeochemical model implementation has a parametrization of pH and elements of the carbonate system
(
$\text{CO}_2$,
$\text{HCO}_3^-$,
$\text{CO}_3^{2-}$
), which are introduced as diagnostic variables.
Also, the set of equilibrium constants are parametrized as diagnostic variables.
Diagnostic variables are the functions of some environment conditions (temperature, salinity, etc.), state variables, other diagnostic variables.
Their values do not depend on their previous values.
For example, $[\text{CO}_2]$ is calculated each time step as a function of the $[\text{DIC}]$ state variable, the first and second dissociation constants of carbonic acid, and pH.

To estimate $\text{CO}_2$ atmosphere-seawater exchange dynamics the biogeochemical model does the following routines.
At first, the number of equilibrium constants are calculated as functions of temperature, salinity, and applied pressure ([Zeebe and Wolf-Gladrow, 2001]).
Then the total scale pH is calculated following [Munhoven, 2013] (it is a function of the equilibrium constants and some state variables).
According to pH and equilibrium constants concentrations of carbonate system elements are recovered.
Finally, the partial pressure of carbon dioxide $\text{pCO}_2 = \frac{[\text{CO}_2]}{K_0}$ where $K_0$ is Henry's constant (see [Zeebe and Wolf-Gladrow, 2001]) is calculated.
Air–sea exchange of $\text{CO}_2$ is calculated following [Yakushev et al., 2017] as a function of $\text{pCO}_2$ in seawater (it is what we calculated previously), $\text{pCO}_2$ in air, and wind speed.
The values of $\text{pCO}_2$ in air and wind speed are constant for entire year (390 $\mu \text{M M}^{-1}$ and 5 $\text{m} \cdot \text{s}^{-1}$ correspondingly). 

[Zeebe and Wolf-Gladrow, 2001]: https://www.elsevier.com/books/co2-in-seawater-equilibrium-kinetics-isotopes/zeebe/978-0-444-50946-8
[Munhoven, 2013]: https://gmd.copernicus.org/articles/6/1367/2013/gmd-6-1367-2013.html
[Yakushev et al., 2017]: https://doi.org/10.5194/gmd-10-453-2017

#### Setup

For our calculations we consider the Wadden Sea - the southern North Sea area.
For the modelling domain we take a 10 km × 10 km square of the depth of 20 m (it is approximately a mean depth of the German Bight according to [Becker et al. (1992)] and <https://www.ngdc.noaa.gov/mgg/global/>) representing the area in the sourthern North Sea at the border with the Wadden Sea.
We label it Box A.
It has 5 layers: 0-2.5 m, 2.5-5 m, 5-10 m, 10-15 m, 15-20 m.
Box A (10 km × 10 km × 20 m) is connected to
1. Box B - the box (10 km × 10 km × 2.5 m) representing the adhering Wadden Sea area. Additional fluxes of TA and DIC can come from this box to the surface (0-2.5 m) layer of the Box A. It consists of a single layer.
2. Box C - two same size (10 km × 10 km × 20 m) boxes with the similar structure from the 2 opposite sides of Box A with which Box A can exchange with nutrients, DIC and TA by horizontal diffusion. So it represents the surrounding southern North Sea. Two boxes are necessary for calculations only (to calculate a second derivative, we need three points). One can consider these boxes as a single box - Box C. 

All calculations occur in Box A only.
Other boxes are sources of nutrients, TA, and DIC in some scenarios (see below).
The modelling setup scheme is depicted in Fig. M1.
The transport model each time step takes state variables increments from the biogeochemical model and updates concentrations of these variables due to exhange between layers of Box A and between Box A and other boxes. 

[Becker et al. (1992)]: https://www.jstor.org/stable/24832635

![Image](misc/mod_domain.png "mod_domain")

**Figure M1.** The modelling setup scheme.
There are actually no gaps between the boxes, it is for sake of clarity. 

TA and DIC fluxes from Box B to Box A are defined to fit a year profile reported by [Thomas et al. (2009)], see Fig. M2.
The fluxes present in several scenarios (see below); one can find the calculations in [s_2_generate_netcdf.ipynb](s_2_generate_netcdf.ipynb).
They do not pretend to represent the real TA and DIC transport from the Wadden Sea to the North Sea, but reproduce some important features.
The absolute value of yearly TA import equals 42 GM, which is the same order of previous estimations [Thomas et al. (2009)], [Schwichtenberg (2013)].
The absolute value of yearly DIC import is 92 GM, which approximately fits values provided by [Brasse et al. (1999)] after rescaling it to year.
TA to DIC ratio falls into the previously reported range [Schwichtenberg (2013)].
Also, as mentioned previously TA input seasonality fits the reported one by [Thomas et al. (2009)].

[Thomas et al. (2009)]: https://doi.org/10.5194/bg-6-267-2009
[Yakubov]: https://github.com/limash/Alkalinity_in_the_Wadden_Sea
[Schwichtenberg (2013)]: https://pure.mpg.de/rest/items/item_2245755/component/file_2245756/content
[Brasse et al. (1999)]: https://doi.org/10.1016/S1385-1101(99)00020-9

![Image](misc/M_TA_inflow.png "TA_inflow")
![Image](misc/M_DIC_inflow.png "TA_inflow")

**Figure M2.** TA and DIC inflow year profiles from Box B to Box A, which appear in some scenarios.

Forcing (temperature, salinity, density, photosynthetically active radiation), initial conditions for state variables, and concentrations of nutrients in Box C are from the World Ocean Database.
The data are averaged to one year for the years 2000 - 2010 from a rectangular region (the Southern North Sea, 54.35-55.37$^{\circ}$N 6.65-8.53$^{\circ}$E) that is adjacent to the North Frisian Wadden Sea.
The vertical diffusion coefficients in the water column (Box A) are calculated according to the vertical density distributions following [Gargett (1984)].

[Gargett (1984)]: https://doi.org/10.1357/002224084788502756

The horizontal diffusion coefficient used to calculate horizontal diffusion between Box A and Box C (150 m$^2$ sec$^{-1}$) is an apparent diffusivity taken from [Okubo (1971)] which is approximately the average value measured for the North Sea area with the correction to the scale-dependent diffusivity from [Okubo (1976)].
The corresponding horizontal scale (which is according to [Okubo (1971)] the distance where 95% of tracer remains within if the distribution of the tracer is radially symmetric and Gaussian) to this diffusion coefficient is approximately twice bigger than the side of the modelling domain.
The concentrations in the adhering to the modelling domain area (Box C) are permanent.

[Okubo (1971)]: https://doi.org/10.1016/0011-7471(71)90046-5
[Okubo (1976)]: https://doi.org/10.1016/0011-7471(76)90897-4

#### Scenarios

All scenarios are run till the steady state repeating one year.
One can find all and explore the resulting data of any scenario in [s_2a_model_output.ipynb](s_2a_model_output.ipynb).

At first, we calculate $\text{pCO}_2$ values and atmosphere-seawater exchange of $\text{CO}_2$ in Box A without allochthonous TA and DIC influxes from Box B, but taking into account the exchange by nutrients with Box C.
We refer it as a 'Baseline' scenario (**S-baseline**).
There is no exhange of DIC and TA with Box C in this scenario; the values of DIC and TA in the Box C for other scenarios are from this scenario.
This scenario represents the behaviour of the biogeochemical system of the German Bight without the Wadden Sea influence.

The second scenario (**S-TA_DIC_inflow**) takes into account both TA and DIC fluxes from the Box B and TA, DIC, and nutrients horizontal diffusive exchange with Box C.
This scenario helps us to identify how allochthonous alkalinity and DIC from the Wadden Sea can change the $\text{CO}_2$ atmosphere-seawater exchange in the German Bight.

The third scenario (**S-TA_DIC_zero-average**) is similar to the second one with the difference that the concentrations of TA and DIC in the Wadden Sea are normalized to the average North Sea values from the baseline scenario (**S-baseline**).
From these averaged values TA and DIC fluxes from Box B to Box A are calculated.
With the same average values there is no additional alkalinity and DIC per year comes from Box B to Box A (so the resulting yearly flux is zero, but the shape of profiles are similar to depicted in Fig. M2).
This case helps to establish if pure TA and DIC seasonality can change the $\text{CO}_2$ atmosphere-seawater exchange without any additional input of TA and DIC.

The fourth scenario (**S-DIC**) takes into account only DIC flux from Box B and DIC exchange with Box C without considering the flux and exchange of TA.
It represents the case if for example all OM in the Wadden Sea degrades due to aerobic processes with no alkalinity generation and with no other alkalinity generating processes.
Using this scenario we can evaluate the effect the Wadden Sea causes to the carbon dioxide atmosphere-seawater exchange due to its specific properties.

The next two scenarios **S-noTEMP_noinflow** and **S-noTEMP_inflow** are supposed to reveal biological contribution to $\text{pCO}_2$ control.
For that purpose the temperature is hardcoded to be equal to year mean value of 10.2 $^\circ \text{C}$ for equilibrium constants calculations and the air–sea exchange of $\text{CO}_2$ calculations.
The difference between two scenarios is that **S-noTEMP_inflow** contains TA and DIC fluxes from the Box B to Box A, so we can estimate if this inflow dominates over biological effect or not.

The last two scenarios **S-noBIO_noinflow** and **S-noBIO_inflow** reveal the temperature effect on $\text{CO}_2$ exchange between seawater and the atwosphere.
The primary production is supressed in these scenarios to show the effect of temperature.
Again, to compare temperature effect with influence of advective input of DIC and TA there are fluxes of these elements in **S-noBIO_inflow** run.

**Table M1.** The description of the scenarios.
TA and DIC fluxes are yearly.

| Scenario | TA flux | DIC flux | TA exchange | DIC exchange | Temperature | Biology |
|----------|---------|----------|-------------|--------------|-------------|---------|
| **S-baseline** | 0 | 0 | no | no | yes | yes |
| **S-TA_DIC_inflow** | 42 GM | 92 GM | yes | yes | yes | yes |
| **S-TA_DIC_zero_average** | 0 | 0 | yes | yes | yes | yes |
| **S-DIC** | 0 | 92 GM | no | yes | yes | yes |
| **S-noTEMP_noinflow** | 0 | 0 | no | no | no | yes |
| **S-noTEMP_inflow** | 42 GM | 92 GM | no | no | no | yes |
| **S-noBIO_noinflow** | 0 | 0 | no | no | yes | no |
| **S-noBIO_inflow** | 42 GM | 92 GM | no | no | yes | no |